## Beispiel wie man code aus einem String ausführen kann

In [ ]:
code_to_execute = "print('Hello, World!')"

try:
    exec(code_to_execute)
except Exception as e:
    print(f"An error occurred: {e}")

## Following steps are needed for testing:
1. Identify code from agent output string (ask agent to respond only with code)
2. Save code in dedicated variable 
3. Compare code output with ground trouth output defined by researcher

Before setting up the testing framework like this, we have to know which functions we want to test because this determines the ground truth ouput. Therefore the research for which functions to use should happen before setting up the framework @Felix

See below for sample test walkthrough:

In [ ]:
import pandas as pd

agent_input = "How can I calculate the mean of all the values this list: [1, 2, 3, 4, 5] in Python? Please answer only with the code. Save the result in a variable called 'result'."

agent_output = """
my_list = [1, 2, 3, 4, 5]
result = sum(my_list) / len(my_list)
print(result)
"""

code = agent_output # for this example this is not needed, however if the agent ouput requires extraction this should happen before this statement

desired_result = 3.0 # to be defined by the researcher

def test_my_function(code, desired_result):
    local_vars = {}
    exec(code, globals(), local_vars)
    agent_result = local_vars.get('result', None)

    if isinstance(agent_result, pd.DataFrame):
        if desired_result.equals(agent_output):
            print("Agent output was correct.")
        else:
            print("Agent output was not correct.")

    else:
        if agent_result == desired_result:
            print("Agent output was correct.")
        else:
            print('Agent output was not correct.')

test_my_function(code, desired_result)

## Testing this with sample pandas function ".from_dummies()"

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})

desired_result = pd.from_dummies(df, sep="_")

agent_input = """
How can I convert this dataframe: df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]}) into a categorical dataframe,
using only pandas. Please only respond with the code. Use only pandas and the standard libraries. Save the result dataframe in a variable called "result".
"""

agent_output = """
import pandas as pd

df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})

result = df.astype("category")
"""


test_my_function(agent_output, desired_result)

## Testing with agent

In [ ]:
from llms.agents.react import ReActAgent
from llms.clients.gpt import GPTClient
from llms.settings import settings

client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id="gpt-4-32k",
    max_response_tokens=1000,
    temperature=0.0,
)
agent = ReActAgent(client)

# Get the user's order
user_prompt = """
How can I convert this dataframe: df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]}) into a categorical dataframe?
"""

final_answer = agent.run(user_prompt)

In [ ]:
final_answer # response function

In [ ]:
# Create a namespace for the execution
namespace = {}

exec(final_answer, namespace)

response_function = namespace['response_function']

sample_df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})
result_df = response_function(sample_df)

print(result_df)

## Building testing_framework

In [ ]:
import pandas as pd
from tests.pandas import TEST_CASES
from llms.agents.react import ReActAgent
from llms.clients.gpt import GPTClient
from llms.settings import settings

client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id="gpt-4-32k",
    max_response_tokens=1000,
    temperature=0.0,
)
agent = ReActAgent(client)

test_results = []

for test_case in TEST_CASES:
    agent_error = None # variable to store errors that the agent code produces

    # get response function from agent
    final_answer = agent.run(test_case.prompt)
    namespace_agent = {}
    exec(final_answer, namespace_agent)
    response_function = namespace_agent['response_function']

    # get desired result and save it in a variable called data
    data_string = test_case.data
    local_vars = {}
    exec(data_string, globals(), local_vars)

    # retrieve the correct function
    correct_function_string = test_case.correct_function
    namespace_correct = {}
    exec(correct_function_string, namespace_correct)
    correct_function = namespace_correct['correct_function']

    # execute the correct function with the data as parameter and save it as desired result
    desired_result = correct_function(*[local_vars.get(arg, None) for arg in local_vars])

    # execute the agent function with the data as parameter and save it as agent_result, store error, if agent code produces an error
    try:
        agent_result = response_function(*[local_vars.get(arg, None) for arg in local_vars])
    except Exception as e:
        agent_result = None
        agent_error = e

    # this has to be extended, each time we expect another data type as the desired output
    if isinstance(desired_result, pd.DataFrame) or isinstance(desired_result, pd.Series) or isinstance(desired_result, pd.Index):
        if desired_result.equals(agent_result):
            print(f"Agent output was correct for test case.") # add test_case.id
            test_results.append({'result': f"correct for test case", 'agent_result': agent_result, 'desired_result': desired_result, 'agent_error': agent_error})
        else:
            print(f"Agent output was not correct for test case.")
            test_results.append({'result': f"false for test case", 'agent_result': agent_result, 'desired_result': desired_result, 'agent_error': agent_error})

    else:
        if agent_result == desired_result:
            print(f"Agent output was correct for test case.")
            test_results.append({'result': f"correct for test case", 'agent_result': agent_result, 'desired_result': desired_result, 'agent_error': agent_error})
        else:
            print(f"Agent output was not correct for test case.")
            test_results.append({'result': f"false for test case ", 'agent_result': agent_result, 'desired_result': desired_result, 'agent_error': agent_error})

In [ ]:
test_results

In [ ]:
import json

for result in test_results:
    if 'agent_result' in result and isinstance(result['agent_result'], pd.DataFrame):
        result['agent_result'] = result['agent_result'].to_dict(orient='records')
    if 'desired_result' in result and isinstance(result['desired_result'], pd.DataFrame):
        result['desired_result'] = result['desired_result'].to_dict(orient='records')

def convert_datetime(obj):
    if isinstance(obj, (pd.Timestamp)):
        return obj.isoformat()

# Specify the JSON file path
json_file_path = 'output.json'

# Write the data to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(test_results, json_file, default=convert_datetime, indent=2)

# Experimentation

In [ ]:
import pandas as pd
import numpy as np
def correct_function(*args):
    result = data_1.div(data_2, fill_value=0)
    return result

import numpy as np
data_1 = pd.Series([1, 1, 1, np.nan], index=['a', 'b', 'c', 'd'])
data_2 = pd.Series([1, np.nan, 1, np.nan], index=['a', 'b', 'd', 'e'])

xzy = []
correct_function(data_1, data_2, xzy)